In [1]:
!pip install datasets
!pip install torch torchvision torchaudio
!pip install transformers
!pip install rouge-score nltk
!pip install sentencepiece
!pip install evaluate

You should consider upgrading via the '/home/marie/newenv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/marie/newenv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/marie/newenv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/marie/newenv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/marie/newenv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/marie/newenv/bin/python3 -m pip install --upgrade pip' command.


In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/marie/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
from datasets import load_dataset, load_metric
from transformers import (
    DataCollatorForSeq2Seq,
    MT5ForConditionalGeneration,
    MT5Tokenizer,
    pipeline,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
) 
import evaluate
import numpy as np
import datasets

In [10]:
tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")  # cointegrated/rut5-small google/mt5-small
model = MT5ForConditionalGeneration.from_pretrained('google/mt5-small')

# tokenizer = MT5Tokenizer.from_pretrained("results/checkpoint-121500")  # это если мы хотим подгрузить веса модели которую уже обучали
# model = MT5ForConditionalGeneration.from_pretrained('results/checkpoint-121500')

In [11]:
max_target_tokens_count = 128
max_source_tokens_count = 1024


def preprocess_function(examples):
    inputs = examples["text"]
    model_inputs = tokenizer(inputs, max_length=max_source_tokens_count, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=max_target_tokens_count, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs

In [7]:
dataset = load_dataset('IlyaGusev/gazeta',revision="v2.0")
tokenized_dataset = dataset.map(preprocess_function, batched=True)

No config specified, defaulting to: gazeta/default
Found cached dataset gazeta (/home/marie/.cache/huggingface/datasets/IlyaGusev___gazeta/default/2.0.0/c329f0fc1c22ab6e43e0045ee659d0d43c647492baa2a6ab3a5ea7dac98cd552)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/marie/.cache/huggingface/datasets/IlyaGusev___gazeta/default/2.0.0/c329f0fc1c22ab6e43e0045ee659d0d43c647492baa2a6ab3a5ea7dac98cd552/cache-dcff2e3e8af4e408.arrow
Loading cached processed dataset at /home/marie/.cache/huggingface/datasets/IlyaGusev___gazeta/default/2.0.0/c329f0fc1c22ab6e43e0045ee659d0d43c647492baa2a6ab3a5ea7dac98cd552/cache-383555cb89ad5c49.arrow
Loading cached processed dataset at /home/marie/.cache/huggingface/datasets/IlyaGusev___gazeta/default/2.0.0/c329f0fc1c22ab6e43e0045ee659d0d43c647492baa2a6ab3a5ea7dac98cd552/cache-123aab2cbcbfa8b3.arrow


In [12]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [13]:
rouge = load_metric("rouge")
bleu = evaluate.load("bleu")

<ipython-input-13-9921d770c86d>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")


In [14]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = {}
    result_rouge = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    # Extract a few results
    result.update({key: value.mid.fmeasure * 100 for key, value in result_rouge.items()})
    
    result_bleu = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    # Extract a few results
    result["bleu"] = result_bleu["bleu"] * 100
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    result["char_len"] = np.mean([len(t) for t in decoded_preds])
    
    return {k: round(v, 4) for k, v in result.items()}

In [15]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/NLP/results",
    evaluation_strategy="steps",
    eval_steps=25,
    logging_steps=25,
    learning_rate=4e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    fp16=False,
    predict_with_generate=True,
    generation_max_length=max_target_tokens_count,
    generation_num_beams=5
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()